In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
#Count the number of images in the respective classes - 0 - Brain Tumor and 1 - Healthy
ROOT_DIR = "/content/drive/MyDrive/Dataset/Brain_Tumor_Detection"
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))
number_of_images.items()


In [ ]:
len(os.listdir("/content/drive/MyDrive/Dataset/Brain_Tumor_Detection"))

In [5]:
def dataFolder(p, split):
  #Creating train Folder
  if not os.path.exists('./'+p):
    os.mkdir('./' + p)

    for dir in os.listdir(ROOT_DIR):
      os.makedirs('./' + p + "/" +dir)

      for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)),
                                size = (math.floor(split * number_of_images[dir])-5),
                                replace = False):
        O = os.path.join(ROOT_DIR, dir, img)
        D = os.path.join("./"+p , dir)
        shutil.copy(O, D)
        os.remove(O)
  else:
    print(f"{p} folder exists")


Creating Folders


In [6]:
dataFolder('train', 0.7)

In [7]:
dataFolder('val', 0.15)

In [8]:
dataFolder('test', 0.15)

In [ ]:
import os

folder_path = '/content/train/Brain_Tumor'
image_files = os.listdir(folder_path)

# Print the first 5 image filenames
print(image_files[:5])


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np


img_path = os.path.join(folder_path, image_files[0])


img = image.load_img(img_path)

img_array = np.array(img)

print(img_array.shape)


In [11]:
#Model Building

from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras

In [ ]:
#CNN Model

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', input_shape = (224, 224, 3)))

model.add(Conv2D(filters = 36, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.summary()

In [23]:
model.compile(optimizer = 'adam', loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [14]:
def preprocessingImages(path):
  """
  input : path
  output: preprocessed images
  """
  image_data = ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2, rescale = 1/255, horizontal_flip = True)
  image = image_data.flow_from_directory(directory = path, target_size = (224, 224), batch_size = 32, class_mode = 'binary')
  return image


In [ ]:
path = '/content/train'
train_data = preprocessingImages(path)

In [16]:
def preprocessingImages2(path):
  """
  input : path
  output: preprocessed images
  """
  image_data = ImageDataGenerator(rescale = 1/255)
  image = image_data.flow_from_directory(directory = path, target_size = (224, 224), batch_size = 32, class_mode = 'binary')
  return image

In [ ]:
path = '/content/test'
test_data = preprocessingImages2(path)

In [ ]:
path = '/content/val'
val_data = preprocessingImages2(path)

In [ ]:
train_data.class_indices

In [39]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 3, verbose = 1, mode ='auto')

mc = ModelCheckpoint(monitor = 'val_accuracy', filepath ='/content/best_model.keras', verbose = 1,mode = 'auto')

cd = [es, mc]

In [ ]:
hs = model.fit(train_data,
                         steps_per_epoch = 8,
                         epochs = 10,
                         verbose = 1,
                         validation_data = val_data,
                         validation_steps = 16)

In [45]:
model.save('/content/best_model.keras')


In [ ]:
#Model Graphical Interpretation

h = hs.history
h.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hs.history['accuracy'])
plt.plot(hs.history['val_accuracy'] , c= 'red')

plt.title('acc vs Val_Acc')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'] , c= 'red')

plt.title('loss vs Val_loss', c = 'red')
plt.show()

In [46]:

from keras.models import load_model

model = load_model('/content/best_model.keras')

In [ ]:
#Model Accuracy

acc = model.evaluate(test_data)[1]
print(f"The accuracy of our model is: {acc * 100} %")

In [48]:
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
path = "/content/train/Brain_Tumor/y0.jpg"
img = load_img(path, target_size = (224, 224))
input_arr = img_to_array(img)/255

input_arr.shape

plt.imshow(input_arr)
plt.show()

input_arr = np.expand_dims(input_arr, axis = 0)

pred = np.argmax(model.predict(input_arr), axis = -1)[0]

if pred == 0:
  print("The MRI image has a Brain Tumor")
else:
  print("The MRI image is a healthy brain.")